# Analysis of mass training of models

### Combine all results into one dataframe

In [9]:
import os
import pandas as pd
import plotly.graph_objects as go
import numpy as np

In [10]:
training_results = "./TRAINING_RESULTS"

image_files_FINAL  = [file[12:-4] for file in os.listdir(f"{training_results}")]
image_files_FINAL

['DeepLabV3Plus__resnet101__JaccardLoss__Freeze_encoder_False',
 'DeepLabV3Plus__resnet34__JaccardLoss__Freeze_encoder_False',
 'DeepLabV3Plus__resnet34__JaccardLoss__Freeze_encoder_False_OLD',
 'FPN__resnet34__JaccardLoss__Freeze_encoder_False',
 'Linknet__resnet34__JaccardLoss__Freeze_encoder_False',
 'MAnet__resnet34__JaccardLoss__Freeze_encoder_False',
 'PAN__resnet34__JaccardLoss__Freeze_encoder_False',
 'PSPNet__resnet34__JaccardLoss__Freeze_encoder_False',
 'UnetPlusPlus__resnet34__JaccardLoss__Freeze_encoder_False',
 'Unet__resnet34__JaccardLoss__Freeze_encoder_False']

# Combine into one dataframe

In [11]:
big_df: pd.DataFrame = None

for f in image_files_FINAL:
    architecture, encoder, loss, encoder_freeze = f.split('__')
    df = pd.read_csv(f"./TRAINING_RESULTS/best_model__{architecture}__{encoder}__{loss}__{encoder_freeze}.csv", index_col=0)
    df['architecture'] = architecture
    df['encoder'] = encoder
    df['loss_func'] = loss
    if type(big_df) == 'NoneType':
        big_df = df
    else:
        big_df = pd.concat((big_df, df))

big_df['memory_used'] /= (1024*1024)

In [12]:
l = big_df.groupby(['architecture', 'encoder', 'loss_func', 'freeze_encoder'])
for g in l:
    print(g[1].iloc[g[1]['val_iou_score'].idxmax()])
    break

train_iou_score         0.951548
train_accuracy          0.985506
train_fscore            0.975126
train_loss              0.048783
val_iou_score           0.943862
val_accuracy            0.983351
val_fscore              0.971062
val_loss                0.056412
parameters              45663441
memory_used              11068.0
freeze_encoder             False
inference_time          0.015183
architecture       DeepLabV3Plus
encoder                resnet101
loss_func            JaccardLoss
Name: 15, dtype: object


In [13]:
big_df[:5]

,train_iou_score,train_accuracy,train_fscore,train_loss,val_iou_score,val_accuracy,val_fscore,val_loss,parameters,memory_used,freeze_encoder,inference_time,architecture,encoder,loss_func
0,0.877840,0.960426,0.933468,0.148276,0.911567,0.973210,0.953552,0.092309,45663441,11068.0,False,0.015183,DeepLabV3Plus,resnet101,JaccardLoss
1,0.907052,0.971414,0.950997,0.096140,0.906916,0.972154,0.951093,0.094688,45663441,11068.0,False,0.015183,DeepLabV3Plus,resnet101,JaccardLoss
2,0.920570,0.975768,0.958506,0.081223,0.916987,0.974587,0.956595,0.084375,45663441,11068.0,False,0.015183,DeepLabV3Plus,resnet101,JaccardLoss
3,0.926813,0.977791,0.961884,0.074451,0.924283,0.977538,0.960526,0.076930,45663441,11068.0,False,0.015183,DeepLabV3Plus,resnet101,JaccardLoss
4,0.930461,0.978968,0.963867,0.070552,0.931390,0.979489,0.964402,0.069466,45663441,11068.0,False,0.015183,DeepLabV3Plus,resnet101,JaccardLoss


# Plot

In [14]:
def plot_data(big_df: pd.DataFrame, y_volumn, y_column_name, plot_title, x_column="index", x_column_name="Epochs", filter=None):
    fig = go.Figure()
    big_df_copy = big_df.copy(deep=True)
    if filter != None:
        big_df_copy.query(filter, inplace=True)
    # for f in image_files_FINAL:
    for data in big_df_copy.groupby(['architecture', 'encoder', 'loss_func', 'freeze_encoder']):
        architecture, encoder, loss, freeze = data[0]
        # fig.add_trace(go.Scatter(x=df.index+1, y=df['train_iou_score'], name=f"{architecture} {encoder} {loss}", hoverinfo='text+name',)) # 
        # fig.add_trace(go.Scatter(x=data[1].index+1, y=data[1]['val_iou_score'], name=f"{architecture} {encoder} {loss}", hoverinfo='y+text+name',)) # 
        hovertemplate=f"<b>{architecture} {encoder} {loss} {freeze}</b><br><br>" + \
                        f"{plot_title}"+": %{y:.3}<br>" + \
                        f"{x_column_name}"+": %{x}<br>" + \
                        "<extra></extra>",
        if x_column == "index":
            fig.add_trace(go.Scatter(x=data[1].index+1, y=data[1][y_volumn], name=f"{architecture} {encoder} {loss} {freeze}", hovertemplate=hovertemplate)) # 
        else:
            fig.add_trace(go.Scatter(x=data[1][x_column], y=data[1][y_volumn], name=f"{architecture} {encoder} {loss} {freeze}", hovertemplate=hovertemplate, mode='markers')) # 
        
    # fig.update_traces(hoverinfo='text+name', mode='lines+markers')
    fig.update_layout(legend=dict(y=0.5, traceorder='reversed', font_size=16))
    fig.update_layout(
        title=f"{plot_title} - (Architecture Encoder Loss)",
        # title="Training loss - (Architecture Encoder Loss)",
        xaxis_title=x_column_name,
        yaxis_title=y_column_name,
            font=dict(
            size=14,
        )
    )
    # fig.update_layout(hovermode="x unified")
    fig.show()

In [15]:
# Loss
plot_data(big_df, y_volumn='train_loss', y_column_name="Jaccard Loss", plot_title="Train loss", filter='loss_func == "JaccardLoss"')
plot_data(big_df, y_volumn='val_loss', y_column_name="Jaccard Loss", plot_title="Val loss", filter='loss_func == "JaccardLoss"')
# IoU
plot_data(big_df, y_volumn='train_iou_score', y_column_name="IoU", plot_title="Train IoU", filter='loss_func == "JaccardLoss"')
plot_data(big_df, y_volumn='val_iou_score',  y_column_name= "IoU", plot_title="Val IoU", filter='loss_func == "JaccardLoss"')

# Memory consumption

In [16]:
def plot_data_memory(big_df: pd.DataFrame, y_volumn, y_column_name, plot_title, x_column="index", x_column_name="Epochs", filter=None):
    fig = go.Figure()
    big_df_copy = big_df.copy(deep=True)
    if filter != None:
        big_df_copy.query(filter, inplace=True)
    # for f in image_files_FINAL:
    for data in big_df_copy.groupby(['architecture', 'encoder', 'loss_func', 'freeze_encoder']):
        architecture, encoder, loss, freeze = data[0]
        df: pd.DataFrame = data[1]
        max_row = df.iloc[df['val_iou_score'].idxmax()]

        hovertemplate=f"<b>{architecture} {encoder} {loss} {freeze}</b><br><br>" + \
                        f"{plot_title}"+": %{y:.3}<br>" + \
                        f"{x_column_name}"+": %{x}<br>" + \
                        "<extra></extra>",
        
        fig.add_trace(go.Scatter(x=[max_row[x_column]], y=[max_row[y_volumn]], name=f"{architecture} {encoder} {loss} {freeze}", hovertemplate=hovertemplate, mode='markers')) # 
        
    # fig.update_traces(hoverinfo='text+name', mode='lines+markers')
    fig.update_layout(legend=dict(y=0.5, traceorder='reversed', font_size=16))
    fig.update_layout(
        title=f"{plot_title} - (Architecture Encoder Loss EncoderFreeze)",
        # title="Training loss - (Architecture Encoder Loss)",
        xaxis_title=x_column_name,
        yaxis_title=y_column_name,
            font=dict(
            size=14,
        )
    )
    # fig.update_layout(hovermode="x unified")
    fig.show()

# plot_data_memory(big_df, y_volumn='train_iou_score', y_column_name="IoU", x_column="memory_used", x_column_name="Memory (mb)", plot_title="Train IoU")
big_df_copy = big_df.copy(deep=True)
big_df_copy['inference_time'] = big_df_copy['inference_time']*1000
plot_data_memory(big_df_copy, y_volumn='val_iou_score', y_column_name="Val IoU", x_column="inference_time", x_column_name="Inference time (ms)", plot_title="Val IoU vs Inference time")
plot_data_memory(big_df, y_volumn='val_iou_score', y_column_name="Val IoU", x_column="memory_used", x_column_name="Memory (mb)", plot_title="Val IoU vs GPU memory")